# 15wk-1: Plotly – `px`를 활용한 시각화

최규빈  
2023-12-04

# 1. 강의영상

In [1]:
pass

# 2. Imports

In [5]:
import numpy as np
import pandas as pd
#---#
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots

In [6]:
pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

# 3. 아이스크림을 많이 먹으면 걸리는 병

## A. 자료

In [170]:
np.random.seed(42)
temp=pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/temp.csv').iloc[:,3].to_numpy()
sales = 20 + 2 * temp + np.random.randn(len(temp))*7 # 온도 -> 아이크림판매량
diss1 = 30 + 0.5 * temp + np.random.randn(len(temp))*1 # 온도 -> 소아마비반응수치
diss2 = 30 + 0.15 * sales + np.random.randn(len(temp))*1 # 아이스크림판매량 -> 소아마비반응수치 
df1 = pd.DataFrame({'temp':temp,'diss':diss1,'sales':sales})
df2 = pd.DataFrame({'temp':temp,'diss':diss2,'sales':sales})

## B. Scatter + Line

In [171]:
df1.plot.scatter(
    x='sales',
    y='diss', 
    title='world1: temp --> (diss,sales)',
    trendline = 'ols', # ols ,lowess
)

In [172]:
df1.assign(temp_cut = lambda df: pd.qcut(df.temp,10)).sort_values('temp_cut')\
.plot.scatter(
    x='sales',
    y='diss',
    color='temp_cut',
    trendline='ols',
    title='world1: temp --> (diss,sales)',
)

In [173]:
df2.assign(temp_cut = lambda df: pd.qcut(df.temp,10)).sort_values('temp_cut')\
.plot.scatter(
    x='sales',
    y='diss',
    color='temp_cut',
    trendline='ols',
    title='world2: temp --> sales --> diss',
)

# NCY

In [206]:
df = pd.read_csv("https://raw.githubusercontent.com/guebin/DV2023/main/posts/NYCTaxi.csv")
df = df.assign(
    log_trip_duration = np.log(df.trip_duration),
    pickup_datetime = df.pickup_datetime.apply(pd.to_datetime),
    dropoff_datetime = df.dropoff_datetime.apply(pd.to_datetime),
    dist = np.sqrt((df.pickup_latitude-df.dropoff_latitude)**2 + (df.pickup_longitude-df.dropoff_longitude)**2),
    #---#
    vendor_id = df.vendor_id.map({1:'A',2:'B'})
).assign(
    speed = lambda df: df.dist / df.trip_duration,
    pickup_hour = lambda df: df.pickup_datetime.dt.hour,
    dropoff_hour = lambda df: df.dropoff_datetime.dt.hour,
    dayofweek = lambda df: df.pickup_datetime.dt.dayofweek
)

In [208]:
df

In [211]:
df.columns

In [215]:
tidydata = df 
fig = px.parallel_coordinates(
    data_frame=tidydata,
    color='speed',
    dimensions=['dist','speed']
)
fig

## C.

In [199]:
big =  df1.assign(
    temp_cut = lambda df: pd.qcut(df.temp,10,labels=list('0123456789')),
    sales_cut = lambda df: pd.qcut(df.sales,4,labels=['Q1','Q2','Q3','Q4']),
    diss_cut = lambda df: pd.qcut(df.diss,10)
)
small = big.groupby('diss_cut').agg({'diss':'mean'}).reset_index().rename({'diss':'diss_mean'},axis=1)
tidydata = big.merge(small).query("temp_cut == '8'")
#---#
fig = px.parallel_coordinates(
    data_frame=tidydata,
    color='diss_mean',
    dimensions=['sales','diss','temp']
)
fig

In [200]:
big =  df2.assign(
    temp_cut = lambda df: pd.qcut(df.temp,10,labels=list('0123456789')),
    sales_cut = lambda df: pd.qcut(df.sales,4,labels=['Q1','Q2','Q3','Q4']),
    diss_cut = lambda df: pd.qcut(df.diss,10)
)
small = big.groupby('diss_cut').agg({'diss':'mean'}).reset_index().rename({'diss':'diss_mean'},axis=1)
tidydata = big.merge(small).query("temp_cut == '8'")
#---#
fig = px.parallel_coordinates(
    data_frame=tidydata,
    color='diss_mean',
    dimensions=['sales','temp','diss']
)
fig

In [201]:
big =  df2.assign(diss_cut = lambda df: pd.qcut(df.diss,4)).sort_values('diss_cut')
small = big.groupby('diss_cut').agg({'diss':'mean'}).reset_index().rename({'diss':'diss_mean'},axis=1)
tidydata = big.merge(small)
#---#
fig = px.parallel_coordinates(
    data_frame=tidydata,
    color='diss_mean',
    dimensions=['sales','temp','diss']
)
fig

In [202]:
big =  df1.assign(
    temp_cut = lambda df: pd.qcut(df.temp,4,labels=['Q1','Q2','Q3','Q4']),
    sales_cut = lambda df: pd.qcut(df.sales,4,labels=['Q1','Q2','Q3','Q4']),
    diss_cut = lambda df: pd.qcut(df.diss,4,labels=['Q1','Q2','Q3','Q4'])
)
small = big.groupby('diss_cut').agg({'diss':'mean'}).reset_index().rename({'diss':'diss_mean'},axis=1)
tidydata = big.merge(small).query("temp_cut == 'Q1'")
#---#
fig = px.parallel_categories(
    data_frame=tidydata,
    color='diss_mean',
    dimensions=['sales_cut','diss_cut']
)
fig

In [178]:
big =  df2.assign(
    temp_cut = lambda df: pd.qcut(df.temp,4,labels=['Q1','Q2','Q3','Q4']),
    sales_cut = lambda df: pd.qcut(df.sales,4,labels=['Q1','Q2','Q3','Q4']),
    diss_cut = lambda df: pd.qcut(df.diss,4,labels=['Q1','Q2','Q3','Q4'])
)
small = big.groupby('temp_cut').agg({'temp':'mean'}).reset_index().rename({'temp':'temp_mean'},axis=1)
tidydata = big.merge(small).query("temp_cut == 'Q1'")
#---#
fig = px.parallel_categories(
    data_frame=tidydata,
    color='temp_mean',
    dimensions=['sales_cut','temp_cut','diss_cut']
)
fig

In [149]:
tidydata

In [150]:
tidydata = pd.DataFrame({'temp':temp,'diss':disease,'sales':icecream_sales})\
.assign(
    temp_cut = lambda df: pd.qcut(df.temp,q=4,labels=['Q1','Q2','Q3','Q4']),
    diss_cut = lambda df: pd.qcut(df.diss,q=4,labels=['Q1','Q2','Q3','Q4']),
    sales_cut = lambda df: pd.qcut(df.sales,q=4,labels=['Q1','Q2','Q3','Q4']),
)
display(tidydata)
#---#
px.parallel_categories(
    data_frame=tidydata,
    color='diss',
    dimensions=['temp_cut','diss_cut', 'sales_cut']
)

# 3. px.

In [3]:
data = dict(
    number=[39, 27.4, 20.6, 11, 2],
    stage=["Website visit", "Downloads", "Potential customers", "Requested price", "invoice sent"])
fig = px.funnel(data, x='number', y='stage')
fig.show()

In [5]:
pd.DataFrame(data)

# 3. pie,

## A. `px.pie`

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2021/master/_notebooks/2021-10-25-FIFA22_official_data.csv').loc[:,lambda df: df.isna().mean()<0.5].dropna()

In [7]:
continent_mapping = {
    'Asia': ['Afghanistan', 'Japan', 'Macau', 'Chinese Taipei', 'Indonesia', 'Korea Republic', 'Kazakhstan', 'Kyrgyzstan', 'Bhutan', 'Philippines', 'Syria', 'China PR', 'Oman', 'Guam', 'Vietnam', 'Jordan', 'Palestine', 'Malaysia', 'Hong Kong', 'Korea DPR', 'Lebanon', 'Uzbekistan', 'India','Iraq', 'Iran', 'Saudi Arabia', 'United Arab Emirates','Australia'],
    'Europe': ['Portugal', 'Germany', 'Belgium', 'Netherlands', 'Croatia', 'Spain', 'Austria', 'Italy', 'France', 'Serbia', 'England', 'Poland', 'Ukraine', 'Wales', 'Scotland', 'Czech Republic', 'Slovakia', 'Romania', 'Bosnia and Herzegovina', 'Republic of Ireland', 'Norway', 'Sweden', 'Bulgaria', 'Lithuania', 'Estonia', 'Latvia', 'Liechtenstein','Albania','Denmark','Finland','Greece','Hungary','Iceland','Luxembourg','Northern Ireland','Slovenia','Switzerland','Andorra','Azerbaijan','Belarus','Cyprus','Faroe Islands','Georgia','Kosovo','Malta','Moldova','Montenegro','North Macedonia','Armenia','Gibraltar','Russia','Turkey','Israel'],
    'South America': ['Uruguay', 'Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Paraguay', 'Venezuela', 'Suriname', 'Bolivia','Peru','Guyana'],
    'Africa': ['Egypt', "Côte d'Ivoire", 'Senegal', 'Morocco', 'Ghana', 'Algeria', 'Guinea', 'Mali', 'Congo DR', 'Liberia', 'Cameroon', 'Tunisia', 'Comoros', 'Kenya', 'South Africa', 'Zimbabwe', 'Madagascar', 'Mozambique', 'Equatorial Guinea', 'Congo', 'Burundi', 'Grenada', 'Thailand', 'Togo', 'Sudan', 'Mauritania','Guinea Bissau','Libya','Nigeria','Zambia','Angola','Benin','Burkina Faso','Cape Verde Islands','Central African Republic','Chad','Eritrea','Gabon','Gambia','Mauritius','Namibia','Rwanda','Sierra Leone','South Sudan','São Tomé e Príncipe','Uganda','Niger'],
    'North and Central America': ['Antigua and Barbuda', 'Barbados', 'Belize', 'Bermuda', 'Canada', 'Costa Rica', 'Cuba', 'Curacao', 'Dominican Republic', 'El Salvador', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Montserrat', 'Panama', 'Puerto Rico', 'Saint Kitts and Nevis', 'Saint Lucia', 'Trinidad and Tobago', 'United States'],
    'Oceania': ['New Zealand', 'Fiji', 'Papua New Guinea','New Caledonia'],
}

In [8]:
df.assign(
    Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]
)

In [9]:
tidydata = df.assign(
    Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]
).groupby('Continent').agg({'ID':'count'}).reset_index().rename({'ID':'Count'},axis=1)
px.pie(
    data_frame=tidydata,
    names = 'Continent',
    values = 'Count',
)
#tidydata

`-` pie chart with hole

In [10]:
tidydata = df.assign(
    Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]

).query('Overall > Overall.mean()')\
.groupby('Continent').agg({'ID':'count'}).reset_index().rename({'ID':'Count'},axis=1)
#---#
px.pie(
    data_frame=tidydata,
    labels = 'Continent',
    values = 'Count',
    hole = 0.3
)
#tidydata

`-`

In [11]:
tidydata = df.assign(
    Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]

).query('Overall > Overall.mean()')\
.groupby('Continent').agg({'ID':'count'}).reset_index().rename({'ID':'Count'},axis=1)
#---#
go.Figure(
    data = go.Pie(labels = tidydata.Continent, values= tidydata.Count, pull = [0.2,0,0,0,0,0],hole=0.3)
)

In [12]:
a=list([1,2,3])

## B. `px.sunburst`

In [13]:
tidydata = df.groupby('Nationality').agg('size').reset_index().rename({0:'Count'},axis=1)\
.assign(
    Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]
)
#---# 
px.sunburst(
    data_frame=tidydata, 
    path=['Continent','Nationality'],
    values='Count'
)

## C. `px.treemap`

In [14]:
tidydata = df.groupby('Nationality').agg('size').reset_index().rename({0:'Count'},axis=1)\
.assign(
    Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]
)
#---# 
px.treemap(
    data_frame=tidydata, 
    path=['Continent','Nationality'],
    values='Count'
)

In [28]:
# tidydata = df.groupby('Nationality').agg('size').reset_index().rename({0:'Count'},axis=1)\
# .assign(
#     Continent = lambda df: [k for x in df['Nationality'] for k,v in continent_mapping.items() if x in v]
# )
# #---# 
# px.icicle(
#     data_frame=tidydata, 
#     path=['Continent','Nationality',],
#     values='Count'
# )

# 4.

In [330]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/insurance.csv')
df

In [350]:
_df = df.assign(
    age_cut = lambda df: pd.qcut(df.age,q=4,labels=['Q1','Q2','Q3','Q4']),
    bmi_cut = lambda df: pd.qcut(df.bmi,q=3,labels=['low','midium','high']),
    charges_cut = lambda df: pd.qcut(df.charges,q=4,labels=['Q1','Q2','Q3','Q4']),
)

In [356]:
_df

In [369]:
tidydata = df.assign(
    age_cut = lambda df: pd.qcut(df.age,q=4,labels=['Q1','Q2','Q3','Q4']),
    bmi_cut = lambda df: pd.qcut(df.bmi,q=3,labels=['low','midium','high']),
    charges_cut = lambda df: pd.qcut(df.charges,q=4,labels=['Q1','Q2','Q3','Q4']),
).assign(
    charges_mean = lambda df: df.charges_cut.map(df.groupby('charges_cut').agg({'charges':'mean'}).rename({'charges':'charges_mean'},axis=1).to_dict()['charges_mean'])
)
tidydata

In [371]:
tidydata = df.assign(
    age_cut = lambda df: pd.qcut(df.age,q=4,labels=['Q1','Q2','Q3','Q4']),
    bmi_cut = lambda df: pd.qcut(df.bmi,q=3,labels=['low','midium','high']),
    charges_cut = lambda df: pd.qcut(df.charges,q=4,labels=['Q1','Q2','Q3','Q4']),
).assign(
    charges_mean = lambda df: df.charges_cut.map(df.groupby('charges_cut').agg({'charges':'mean'}).rename({'charges':'charges_mean'},axis=1).to_dict()['charges_mean'])
)
#---#
px.parallel_categories(
    data_frame=tidydata,
    color='charges_mean',
    dimensions=['smoker','age_cut','bmi_cut','region','children','charges_cut']
)

In [373]:
tidydata = df.assign(
    age_cut = lambda df: pd.qcut(df.age,q=4,labels=['Q1','Q2','Q3','Q4']),
    bmi_cut = lambda df: pd.qcut(df.bmi,q=3,labels=['low','midium','high']),
    charges_cut = lambda df: pd.qcut(df.charges,q=4,labels=['Q1','Q2','Q3','Q4']),
).assign(
    charges_mean = lambda df: df.charges_cut.map(df.groupby('charges_cut').agg({'charges':'mean'}).rename({'charges':'charges_mean'},axis=1).to_dict()['charges_mean'])
).query("smoker == 'no'")
#---#
px.parallel_categories(
    data_frame=tidydata,
    color='charges_mean',
    dimensions=['age_cut','bmi_cut','region','children','charges_cut']
)

In [412]:
tidydata = df.assign(
    Fsize = lambda df: df['SibSp'] + df['Parch'],
    Sex = lambda df: df['Sex'].map({'female':0,'male':1}),
).sort_values('Fsize')
px.parallel_categories(
    data_frame=tidydata,
    color='Survived',
    dimensions=['Fsize','Pclass','Embarked','Sex','Survived']
)

In [413]:
tidydata = df.assign(
    Fsize = lambda df: df['SibSp'] + df['Parch'],
    Sex = lambda df: df['Sex'].map({'female':0,'male':1}),
)
px.parallel_categories(
    data_frame=tidydata,
    color='Sex',
    dimensions=['Fsize','Pclass','Embarked','Sex','Survived']
)

In [399]:
df = px.data.wind()
fig = px.bar_polar(df, r="frequency", theta="direction",
                   color="strength", template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()

In [400]:
import plotly.graph_objects as go
import urllib, json

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()